In [1]:
!pip install roboflow
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00


# Notebook for colab training

In [2]:
%cd

/root


In [3]:
ROBOFLOW_API_KEY = "bqQx7aG8EwPnLZVzCCws"

In [4]:
from roboflow import Roboflow
from pathlib import Path
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("tree-detection-v4x2j").project("tree-quality-3")
version = project.version(7)
dataset = version.download("yolov11", location="/root/Data/trees-1")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /root/Data/trees-1 in yolov11:: 100%|██████████| 1748/1748 [00:00<00:00, 5493.77it/s]


In [5]:
project = rf.workspace("govind-rai-aiwsm").project("shrubs")
version = project.version(1)
dataset = version.download("yolov11", location="/root/Data/shrubs-1")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /root/Data/shrubs-1 in yolov11:: 100%|██████████| 1309/1309 [00:00<00:00, 4329.58it/s]


In [6]:
project = rf.workspace("mtlworkspace").project("bush-detection")
version = project.version(7)
dataset = version.download("yolov11", location="/root/Data/shrubs-2")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /root/Data/shrubs-2 in yolov11:: 100%|██████████| 494/494 [00:00<00:00, 5008.05it/s]


In [7]:
project = rf.workspace("mohamed-snoqx").project("tree-6szhy")
version = project.version(1)
dataset = version.download("yolov11", location="/root/Data/trees-2")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /root/Data/trees-2 in yolov11:: 100%|██████████| 3142/3142 [00:01<00:00, 2661.40it/s]


In [8]:
import os
from pathlib import Path
import shutil
import yaml

In [9]:
PATH_TO_SAVE_DATASET = Path("/root/Data")
new_dataset_path = Path("/root/Data/combined_dataset")
new_dataset_path.mkdir(parents=True, exist_ok=True)

In [10]:
for split in ["train", "valid", "test"]:
    (new_dataset_path / split / "images").mkdir(parents=True, exist_ok=True)
    (new_dataset_path / split / "labels").mkdir(parents=True, exist_ok=True)

In [11]:
datasets = [
    {"path": PATH_TO_SAVE_DATASET / "trees-1", "type": "tree"},
    {"path": PATH_TO_SAVE_DATASET / "shrubs-1", "type": "shrub"},
    {"path": PATH_TO_SAVE_DATASET / "shrubs-2", "type": "shrub"},
    {"path": PATH_TO_SAVE_DATASET / "trees-2", "type": "tree"}
]

In [12]:
for ds in datasets:
    ds_path = ds["path"]
    ds_type = ds["type"]
    new_class_id = 0 if ds_type == "tree" else 1  # tree=0, shrub=1

    for split in ["train", "valid", "test"]:
        src_images = ds_path / split / "images"
        src_labels = ds_path / split / "labels"

        # Пропускаем, если папки не существуют
        if not src_images.exists() or not src_labels.exists():
            continue

        # Обрабатываем каждый файл в split
        for img_file in src_images.iterdir():
            base_name = img_file.stem
            ext = img_file.suffix
            new_img_name = f"{ds_path.name}_{base_name}{ext}"
            dest_img = new_dataset_path / split / "images" / new_img_name

            # Копируем изображение
            shutil.copy(img_file, dest_img)

            # Обрабатываем аннотацию
            label_file = src_labels / f"{base_name}.txt"
            if label_file.exists():
                with open(label_file, 'r') as f:
                    lines = f.readlines()

                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if not parts:  # пропускаем пустые строки
                        continue
                    # Заменяем class_id на новый
                    parts[0] = str(new_class_id)
                    new_lines.append(" ".join(parts))

                new_label_name = f"{ds_path.name}_{base_name}.txt"
                dest_label = new_dataset_path / split / "labels" / new_label_name
                with open(dest_label, 'w') as f:
                    f.write("\n".join(new_lines))

In [13]:
data_yaml = {
    "path": str(new_dataset_path),
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": {
        0: "tree",
        1: "shrub"
    }
}

In [14]:
with open(new_dataset_path / "data.yaml", 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

In [15]:
import ultralytics
import yaml
from pathlib import Path
import torch
from datetime import datetime

PATH_TO_SAVE_TRAIN_RUNS = Path("/root/runs")
PATH_TO_YOLO_CONFIG_YAML = Path("/root/config/config.yaml")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [16]:
def create_default_config(path=PATH_TO_YOLO_CONFIG_YAML):
        """
        Создаёт YAML конфиг с базовыми параметрами обучения.
        """
        default_config = {
            "data": str(new_dataset_path / "data.yaml"),
            "epochs": 100,
            "imgsz": 640,
            "batch": 16,  # Уменьши, если OOM
            "device": None,
            "workers": 8,
            "optimizer": "AdamW",  # или "Adam"
            "lr0": 0.001,  # Уменьшен
            "lrf": 0.01,   # Уменьшен
            "momentum": 0.937,
            "weight_decay": 0.0005,
            "warmup_epochs": 3.0,
            "patience": 10,  # Уменьшен
            "resume": False,
            "freeze": 0,  # Убрано замораживание
            "pretrained": True,
            "save": True,
            "save_period": -1,
            "verbose": True,
            "plots": True,
            "project": str(PATH_TO_SAVE_TRAIN_RUNS),
            "name": None,
            "exist_ok": False,
            "cache": False,
            "rect": False,
            "cos_lr": True,  # Включено
            "close_mosaic": 10,
            "amp": True
        }

        if os.path.exists(path):
            print(f"Config is already exists {path}")
        else:
            with open(path, "w") as f:
                yaml.dump(default_config, f)
            print(f"Default config saved to {path}")

In [18]:
%mkdir /root/config/
create_default_config()

Default config saved to /root/config/config.yaml


In [19]:
class YoloWrapper:
    def __init__(self, model_version: str = '11', model_size: str = 'n', finetune: bool = True, device=None, weights_path: str = None):
        """
        model_version: версия YOLO (например, '11')
        model_size: размер модели ('n', 's', 'm', 'l', 'x')
        finetune: True - использовать .pt для дообучения, False - .yaml для обучения с нуля
        device: GPU id (0,1,..) или 'cpu', None - автоматический выбор
        weights_path: путь к зафайнтюненной модели для загрузки
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        if weights_path:

            self.model = ultralytics.YOLO(weights_path)
        else:
            if finetune:
                model_file = "yolo" + f"{model_version}{model_size}.pt"
            else:
                model_file = f"{model_version}{model_size}.yaml"

            self.model = ultralytics.YOLO(model_file)

    def train(self, config_path: str = None, save_dir: str = PATH_TO_SAVE_TRAIN_RUNS, **kwargs):
        """
        Обучение модели.
        - config_path: путь к YAML конфигу с гиперпараметрами
        - save_dir: папка для сохранения результатов (веса, графики, конфиг)
        - kwargs: любые параметры, которые перекрывают YAML конфиг
        """

        config = {}
        if config_path:
            with open(config_path, "r") as f:
                config = yaml.safe_load(f)

        config.update(kwargs)

        if "name" not in config or config["name"] is None:
            config["name"] = datetime.now().strftime("%Y%m%d_%H%M%S")

        if "device" not in config or config["device"] is None:
            config["device"] = self.device

        exp_path = Path(save_dir) / config["name"]
        exp_path.mkdir(parents=True, exist_ok=True)

        config_save_path = exp_path / "config_used.yaml"
        with open(config_save_path, "w") as f:
            yaml.dump(config, f)

        try:
            self.model.train(**config)
        except Exception:
            return None


    def predict(self, source, conf=0.01, imgsz=640, iou=0.25, save=False, save_dir="runs/predict"):
        """
        source: путь к изображению/папке/видео/стриму
        conf: confidence threshold
        imgsz: размер изображения
        save: сохранять ли результаты
        save_dir: папка для сохранения
        """
        try:
            results = self.model.predict(source=source, conf=conf, imgsz=imgsz, iou=iou)
        except Exception:
            return None

        if save:
            save_path = Path(save_dir)
            save_path.mkdir(parents=True, exist_ok=True)
            for i, res in enumerate(results):
                res.save(save_dir=save_path)
        return results

In [20]:
wrapper = YoloWrapper(model_version="11", model_size="m", finetune=True)
wrapper.train(config_path=PATH_TO_YOLO_CONFIG_YAML, batch=32)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/root/Data/combined_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=20250924_1954252, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plo